In [1]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns


C:\Users\rashm\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset=pd.read_csv('yelp_restaurant_train.csv')

print (dataset.head(10))

              business_id                 date               review_id  \
0  HMSDOjt_KCyknzjQ9aI5Jw  2018-11-04 23:44:27  0HWPTEnVT7L5BCKj33erLQ   
1  yXiDD18UV49-7UhA6dWjAw  2018-09-07 20:41:42  Y6xCQlzc9YTXD3IKMGN-oQ   
2  VSNUFYBQ_wOFmRXZ8SeQ4w  2015-09-30 02:07:55  9j_uLXkEgpN52Lz5VJ8sMQ   
3  lVVJMvqu4LXL5rBqjO6eqg  2015-05-30 07:40:17  xilO0UqpI_EYJg0nzTBqzw   
4  EEIz44ewHhOKmfTloAK13g  2017-11-19 16:47:26  P5LDWTH6cxQK-_IWtvyOWw   
5  Huo1lJmVkdqvFuLtA74Nhw  2016-06-28 21:42:10  Jp2QuT-Y0Ms9yNa3Rxch8w   
6  rntKF4Dm9JIJoSNi5XqRsQ  2018-02-22 18:45:25  cSHx_MPeFhPSyV7Oo4QjlQ   
7  jDQIKfsfJU3M8M6n0ESKFg  2018-01-16 00:20:06  i2V-3CJ_a4AMoK5kpWxqVA   
8  qfr7bHu1-VXUTQWXoSMf9g  2011-07-12 14:02:51  gPWOw5SDDzlwVKQfBnF01g   
9  iCQpiavjjPzJ5_3gPD5Ebg  2014-11-05 03:24:39  4O0bln3RzKv4awQ5p6-p9w   

                                                text                 user_id  \
0  I would like to start with the classic line "I...  iGtInQDTZ89mKnkhFWdlfA   
1  We come here for once 

In [3]:
dataset.dtypes

business_id            object
date                   object
review_id              object
text                   object
user_id                object
attributes             object
city                   object
categories             object
avg_business_stars    float64
avg_user_star         float64
stars                 float64
dtype: object

In [4]:
textContentRev = pd.read_csv("yelp_restaurant_train.csv")

textContentRev.drop(["review_id", "city","categories","avg_user_star","attributes","business_id","avg_business_stars","stars"], axis = 1, inplace = True)

revdf = pd.DataFrame(textContentRev)

revdf.head(5)


,date,text,user_id
0,2018-11-04 23:44:27,"I would like to start with the classic line ""I...",iGtInQDTZ89mKnkhFWdlfA
1,2018-09-07 20:41:42,We come here for once at least once a month. I...,O3cItff0mKAfXtl5VmbW2w
2,2015-09-30 02:07:55,First meal in Pittsburgh was courtesy of S&D! ...,U5YQX_vMl_xQy8EQDqlNQQ
3,2015-05-30 07:40:17,The Pork bone soup is delicious! The broth loo...,JrgMipJRhagq42ROTzC_CQ
4,2017-11-19 16:47:26,Words cannot express how delectably divine a d...,8lofUN7rFkwT2bw4b5SM4g


In [5]:
temptable= pd.read_csv("yelp_restaurant_train.csv")
temptable.drop(["review_id", "city","categories","avg_user_star","attributes","text"], axis = 1, inplace = True)


temptable.loc[temptable['stars'] < 3, 'stars'] = -1
temptable.loc[temptable['stars'] > 3, 'stars'] = 1
temptable.loc[temptable['stars'] == 3, 'stars'] = 1

temptable.stars = temptable.stars.astype(int)
temptabledf = pd.DataFrame(temptable)

temptabledf.head(5)



,business_id,date,user_id,avg_business_stars,stars
0,HMSDOjt_KCyknzjQ9aI5Jw,2018-11-04 23:44:27,iGtInQDTZ89mKnkhFWdlfA,4.0,-1
1,yXiDD18UV49-7UhA6dWjAw,2018-09-07 20:41:42,O3cItff0mKAfXtl5VmbW2w,4.0,1
2,VSNUFYBQ_wOFmRXZ8SeQ4w,2015-09-30 02:07:55,U5YQX_vMl_xQy8EQDqlNQQ,4.5,1
3,lVVJMvqu4LXL5rBqjO6eqg,2015-05-30 07:40:17,JrgMipJRhagq42ROTzC_CQ,4.5,1
4,EEIz44ewHhOKmfTloAK13g,2017-11-19 16:47:26,8lofUN7rFkwT2bw4b5SM4g,4.5,1


In [6]:
merger = pd.concat([revdf,temptabledf],axis=1,join="inner")#, join_axes=[df_reviewGraph.user_id])
merger = merger.dropna()

In [7]:
newDS = merger[["stars","text"]] # add date later
newDS.head()

,stars,text
0,-1,"I would like to start with the classic line ""I..."
1,1,We come here for once at least once a month. I...
2,1,First meal in Pittsburgh was courtesy of S&D! ...
3,1,The Pork bone soup is delicious! The broth loo...
4,1,Words cannot express how delectably divine a d...


In [8]:
#Make duplicates of negative samples to balance the data set
newDS1 = pd.concat([newDS[newDS["stars"]==-1],newDS],axis=0)
newDS2 = pd.concat([newDS[newDS["stars"]==-1],newDS1],axis=0)
newDS3 = pd.concat([newDS[newDS["stars"]==-1],newDS2],axis=0)


OverSampledDS = pd.concat([newDS[newDS["stars"]==-1],newDS3],axis=0) 

In [9]:
texts_Consider = []
labels_Consider = []

for i in range(len(OverSampledDS)):
    texts_Consider.append(OverSampledDS.iloc[i,1])
    labels_Consider.append(OverSampledDS.iloc[i,0])

In [10]:
tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(texts_Consider)
sequences = tokenizer.texts_to_sequences(texts_Consider)
data = pad_sequences(sequences, maxlen=600)

In [11]:

model = Sequential()
model.add(Dropout(0.2))
model.add(Embedding(20000, 128, input_length=500))#20000 300
model.add(MaxPooling1D(pool_size=4))
model.add(Conv1D(64, 5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.add(LSTM(128))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data, np.array(labels_Consider), test_size=0.3)

In [13]:
model.fit(data, np.array(labels_Consider), validation_split=0.4, batch_size=200, epochs=1)


Train on 646999 samples, validate on 431333 samples
Epoch 1/1
646999/646999 [==============================] - 28425s 44ms/step - loss: -1.1044 - acc: 0.7725 - val_loss: -1.3196 - val_acc: 0.7829


In [29]:
from sklearn.preprocessing import binarize

p = model.predict(X_test)

type_p = binarize(model_pred,threshold=0.5)

In [30]:
for i in range(len(class_pred)):
    if type_p[i] == 0:
        
        type_p[i] = -1

In [32]:
print("Confusion Matrix for CNN: ")
confusion_matrix(y_test,type_p)

Confusion Matrix for CNN: 


array([[ 44768,   6943],
       [ 17896, 253893]], dtype=int64)